In [ ]:
import numpy as np
from psycopg2 import connect
import pandas as pd
import pm4py
import numpy as np
import pandasql as ps
from pm4py.objects.conversion.log import converter as log_converter
from scipy.stats import variation
from scipy import stats
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.statistics.eventually_follows.log import get as efg_get
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import sklearn.preprocessing as sk
from scipy.stats import chi2_contingency
import math
import statistics
import pingouin as pg
import graphviz
from statsmodels.stats import multitest
from statsmodels.stats.contingency_tables import SquareTable as ST
import copy

In [ ]:
df = pd.read_csv("Logs/MIMIC_Log_Raw.csv")

In [ ]:
df = df.rename({"hadm_id":"case:hadm_id"}, axis=1)

In [ ]:
label_list = ["Dialysis - CRRT", "Hemodialysis", "Non-invasive Ventilation", "Invasive Ventilation"]

In [ ]:
proc_re = df.loc[df["concept:name"].isin(label_list)]

In [ ]:
proc_re["time:timestamp"] = proc_re["time:timestamp"].apply(lambda x: pd.to_datetime(x))
proc_re["endtime"] = proc_re["endtime"].apply(lambda x: pd.to_datetime(x))

In [ ]:
hadms = list(proc_re["case:hadm_id"].unique())

In [ ]:
proc_re = proc_re[["case:hadm_id", "time:timestamp", "endtime", "ordercategoryname", "category", "concept:name"]]

In [ ]:
marker_list = ["Creatinine (serum)", "BUN", "Anion gap", "Hematocrit (serum)", "Heart Rate", "Respiratory Rate", "O2 saturation pulseoxymetry", "Non Invasive Blood Pressure systolic", "Non Invasive Blood Pressure diastolic"]

In [ ]:
meas = df.loc[df["label"].isin(marker_list)]

In [ ]:
meas = meas.sort_values(["case:hadm_id", "time:timestamp"])

In [ ]:
meas_log = meas.copy()

In [ ]:
meas_log["concept:name"] = "Measurement"

In [ ]:
meas_log = meas_log.drop_duplicates(["case:hadm_id", "time:timestamp", "concept:name"])

In [ ]:
meas_log = meas_log.loc[meas_log["case:hadm_id"].isin(hadms)]

In [ ]:
#make endtime represented as second event
proc_re = proc_re.reset_index().drop("index", axis=1)

In [ ]:
rows_list = []

for index, row in proc_re.iterrows():
    r = row
    r = r.drop("time:timestamp", axis="index")
    r = r.rename({"endtime":"time:timestamp"})
    r["concept:name"] = "END " + r["concept:name"]
    rows_list.append(r)

to_concat = pd.DataFrame(rows_list)

In [ ]:
proc_start = proc_re

In [ ]:
proc_start["concept:name"] = "START " + proc_start["concept:name"]

In [ ]:
proc_start.drop("endtime", axis=1, inplace=True)

In [ ]:
proc_start_end = pd.concat([proc_start, to_concat])

In [ ]:
proc_start_end = proc_start_end.sort_values(["case:hadm_id", "time:timestamp"])
proc_start_end = proc_start_end.reset_index().drop("index", axis=1)

In [ ]:
meas_log = meas_log.reset_index().drop("index", axis=1)

In [ ]:
chart_proc_start_end = pd.concat([proc_start_end, meas_log])

In [ ]:
chart_proc_start_end = chart_proc_start_end.sort_values(["case:hadm_id", "time:timestamp"])

In [ ]:
meas["charttime"] = meas["charttime"].apply(lambda x: pd.to_datetime(x))

In [ ]:
#separate into category/value label

In [ ]:
labels = meas["label"].unique()

In [ ]:
#Link measurements to activities
time_from_last_proc_event = pd.to_datetime(0)
time_from_next_proc_event = pd.to_datetime(290, unit="Y")
rows_list = []
for index, row in proc_start_end.iterrows():
    row_to_add = {}
    hadm_id = row["case:hadm_id"]
    act = row["concept:name"]
    if index == len(proc_start_end) - 1:
        time_from_next_proc_event = pd.to_datetime(290, unit="Y")
    elif proc_start_end.iloc[index+1]["case:hadm_id"] == hadm_id:
        time_from_next_proc_event = proc_start_end.iloc[index+1]["time:timestamp"]
    else:
        time_from_next_proc_event = pd.to_datetime(290, unit="Y")
    is_start = "START" in act
    meas_p = meas.loc[meas["case:hadm_id"] == hadm_id]
    if is_start:
        meas_p["time_delta"] = row["time:timestamp"] - meas_p["charttime"]
        meas_p = meas_p.loc[(meas_p["time_delta"] > pd.Timedelta(0)) & (meas_p["charttime"] > time_from_last_proc_event)]
        row_to_add["concept:name"] = "Measurement"
    else:
        meas_p["time_delta"] = meas_p["charttime"] - row["time:timestamp"] 
        meas_p = meas_p.loc[(meas_p["time_delta"] > pd.Timedelta(0)) & (meas_p["charttime"] < time_from_next_proc_event)]
        row_to_add["concept:name"] = "Measurement"
    row_to_add["case:hadm_id"] = hadm_id
    row_to_add["time:timestamp"] = None
    for lab in labels: 
        lab_val = meas_p.loc[meas_p["label"] == lab]
        try:
            closest_val_index = lab_val["time_delta"].idxmin()
            closest_val_row = lab_val.loc[closest_val_index]
            row_to_add[lab] = closest_val_row["valuenum"]
            if row_to_add["time:timestamp"] is None:
                row_to_add["time:timestamp"] = closest_val_row["charttime"]
            if is_start:
                if closest_val_row["charttime"] > row_to_add["time:timestamp"]:
                    row_to_add["time:timestamp"] = closest_val_row["charttime"]
            else: 
                if closest_val_row["charttime"] < row_to_add["time:timestamp"]:
                    row_to_add["time:timestamp"] = closest_val_row["charttime"]
        except:
            row_to_add[lab] = None
    if index == len(proc_start_end) -1:
        time_from_last_proc_event = pd.to_datetime(0)
    elif proc_start_end.iloc[index+1]["case:hadm_id"] == hadm_id:
        time_from_last_proc_event = row["time:timestamp"]
    else:
        time_from_last_proc_event = pd.to_datetime(0)
    if row_to_add["time:timestamp"] is not None:
        rows_list.append(row_to_add)
meas_events = pd.DataFrame(rows_list)

In [ ]:
final_log = pd.concat([proc_start_end, meas_events])

In [ ]:
final_log = final_log.sort_values(["case:hadm_id", "time:timestamp"])

In [ ]:
df = final_log.copy()

In [ ]:
df["time:timestamp"] = df["time:timestamp"].apply(lambda x: pd.to_datetime(x))

In [ ]:
hadms = list(df["case:hadm_id"].unique())

In [ ]:
df = df.sort_values(["case:hadm_id", "time:timestamp"])

In [ ]:
parameters = {log_converter.Variants.TO_EVENT_LOG.value.Parameters.CASE_ID_KEY: 'case:hadm_id'}
event_log = pm4py.format_dataframe(df, case_id='case:hadm_id', activity_key='concept:name', timestamp_key='time:timestamp')
log = pm4py.convert_to_event_log(event_log)

In [ ]:
dfg = dfg_discovery.apply(log)
dfg, sa, ea = pm4py.discover_directly_follows_graph(log)
activities_count = pm4py.get_event_attribute_values(log, "concept:name")

In [ ]:
acts = []
for act in activities_count:
    acts.append(act)
real_acts = copy.deepcopy(acts)
dfr_acts = copy.deepcopy(acts)
dpr_acts = copy.deepcopy(acts)

In [ ]:
dfr_Matrix = pd.DataFrame(columns=dfr_acts, index=dfr_acts)
dpr_Matrix = pd.DataFrame(columns=dpr_acts, index=dpr_acts)

In [ ]:
dfr_activities_count = copy.deepcopy(activities_count)
dpr_activities_count = copy.deepcopy(activities_count)

In [ ]:
#dfr
dfr_dict={}
for act_1 in dfr_activities_count:
    results = list()
    for act_2 in dfr_activities_count:  
        dfr_total = dfg[(act_1, act_2)]
        act_count = activities_count[act_2]
        res = dfr_total/act_count
        dfr_Matrix.loc[act_1, act_2] = res
        results.append(res)
    dfr_dict[act_1] = results
    
        

In [ ]:
#dpr
dpr_dict={}
for act_1 in dpr_activities_count:
    results = list()
    for act_2 in dpr_activities_count:  
        dpr_total = dfg[(act_2, act_1)]
        act_count = activities_count[act_2]
        res = dpr_total/act_count
        dpr_Matrix.loc[act_1, act_2] = res
        results.append(res)
    dpr_dict[act_1] = results
    
        

In [ ]:
df_res = pd.DataFrame(columns=["sum"], index=real_acts)
for act in real_acts:
    l_dfr = list(dfr_Matrix.loc[act])
    l_dpr = list(dpr_Matrix.loc[act])
    df_res.loc[act, "sum"] = (sum(l_dfr) + sum(l_dpr)) / ((len(real_acts)+1) * 2)

In [ ]:
def identify_context(events_to_transform, lambd):
    mapping_before = {}
    mapping_after = {}
    for val in events_to_transform:
        names_before = list(dfr_Matrix.loc[val].index)
        names_after = list(dpr_Matrix.loc[val].index)
        mapping_before[val] = []
        mapping_after[val] = []
        for index,rep_score in enumerate(dfr_Matrix.loc[val]):
            if names_before[index] in events_to_transform or "LacticAcid" in names_before[index]:
                continue
            if rep_score > lambd:
                mapping_before[val].append(names_before[index])
        for index,rep_score in enumerate(dpr_Matrix.loc[val]):
            if names_before[index] in events_to_transform or "LacticAcid" in names_before[index]:
                continue
            if rep_score > lambd:
                mapping_after[val].append(names_after[index])
    return mapping_before, mapping_after

In [ ]:
events_to_transform = ["Measurement"]

In [ ]:
mapping_before, mapping_after = identify_context(events_to_transform, 0.75)

In [ ]:
#End OF Discovery and Context Identification

In [ ]:
########Event Transformation#########

In [ ]:
def transform_event(df, rep_event, rep_mapping_before, rep_mapping_after):
    rows_to_add_intern = []
    row_to_add = {}
    case_ids = list(df["case:hadm_id"].unique())
    for case_id in case_ids:
        df_case = df.loc[df["case:hadm_id"] == case_id]
        #df_case = df_case.reset_index().drop("index", axis=1)
        for index, row in df_case.iterrows():
            if row["concept:name"] == rep_event:
                try:
                    if df_case.loc[index+1]["concept:name"] in rep_mapping_before:
                        row_to_add = row
                        row_to_add["concept:name"] = row_to_add["concept:name"] + " BEFORE " + df_case.loc[index+1]["concept:name"]
                        row_to_add["event_time"] = df_case.loc[index+1]["time:timestamp"]
                        row_to_add["time_diff"] = row_to_add["event_time"] - row_to_add["time:timestamp"]
                        rows_to_add_intern.append(row_to_add)
                    else:
                        pass
                except Exception as e:
                    pass
        
        for index, row in df_case.iterrows():
            if row["concept:name"] == rep_event:
                try:
                    if df_case.loc[index-1]["concept:name"] in rep_mapping_after:
                        row_to_add = row
                        row_to_add["concept:name"] = row_to_add["concept:name"] + " AFTER " + df_case.loc[index-1]["concept:name"]
                        row_to_add["event_time"] = df_case.loc[index-1]["time:timestamp"]
                        row_to_add["time_diff"] = row_to_add["time:timestamp"] - row_to_add["event_time"]
                        rows_to_add_intern.append(row_to_add)
                    else:
                        pass
                except:
                    pass
    rows_to_add_intern = pd.DataFrame(rows_to_add_intern)
    rows_to_add_intern = rows_to_add_intern.sort_values(["case:hadm_id", "time_diff"])
    rows_to_add_intern = rows_to_add_intern.drop_duplicates(["case:hadm_id", "time:timestamp"], keep="first")
    return rows_to_add_intern

In [ ]:
rows_to_add = pd.DataFrame()
df_new_rows = pd.DataFrame()
for rep_event in events_to_transform:
    df_e = df.copy()
    for e in events_to_transform:
        if e != rep_event:
            df_e = df_e.loc[df_e["concept:name"] != e]
    df_e = df_e.sort_values(["case:hadm_id", "time:timestamp"])
    df_e = df_e.reset_index().drop("index", axis=1)
    rows_to_add = transform_event(df_e, rep_event, mapping_before[rep_event], mapping_after[rep_event])
    df_new_rows = pd.concat([df_new_rows, rows_to_add])
new_df = df.copy()
for rep_event in events_to_transform:
    new_df = new_df.loc[new_df["concept:name"] != rep_event]
new_df = pd.concat([new_df, df_new_rows])
new_df = new_df.sort_values(["case:hadm_id", "time:timestamp"])

In [ ]:
new_df.to_csv("Logs/ICU_Log.csv")